In [21]:
import importlib

import numpy as np

import rough_path_signatures_pricing.signature_regressor as sr
import rough_path_signatures_pricing.simulation_models as sm

importlib.reload(sm)
importlib.reload(sr)

<module 'rough_path_signatures_pricing.signature_regressor' from 'C:\\Users\\chris\\VSCodeProjects\\rough-path-signatures-pricing\\src\\rough_path_signatures_pricing\\signature_regressor.py'>

#### GBM Paths for Fitting Linear Functional with Signature to Approximate Payoffs

Create a generator object

In [22]:
gbm_params = {
    "x0": 10,
    "t0": 0,
    "t1": 1,
    "n_steps": 100,
    "seed": 42,
    "mu": 0,
    "sigma": 0.2,
}

In [23]:
gbm_simulator = sm.GBM(**gbm_params)

Creating the pricer object

In [ ]:
N = 5
N_PATHS = 100


def asianPayoff(X: np.ndarray, K: float) -> np.ndarray:
    return np.maximum(np.mean(X, axis=1) - K, 0.0)

In [25]:
gbm_signature_pricer = sr.SignaturePricer(
    simulator=gbm_simulator, func=asianPayoff, signature_degree=N
)


Fit the object to observed prices

In [26]:
PRICE = np.array([100, 110, 120, 140, 150])
STRIKES = np.array([10, 11, 12, 14, 15])

In [27]:
gbm_signature_pricer.fit(PRICE, STRIKES, n_paths=N_PATHS)

Predict price on desired option with given parameter

In [30]:
STRIKE = 13

In [31]:
gbm_signature_pricer.implied_expected_signature_.shape

(364,)

In [32]:
price = gbm_signature_pricer.predict(STRIKE, n_paths=N_PATHS)

In [33]:
price

np.float64(-113652.90088089457)

## Below outdated, was made to simulate prices

#### Creating Basket of Similar Options

In [74]:
hullwhite_params = {"x0": 10, "t0": 0, "t1": 1, "n_steps": 100, "n_paths": 25}
hullwhite_seed = 42


def theta(t):
    return 0.03

In [75]:
hullwhite = sm.HullWhite(a=1, sigma=0.1, theta=theta, seed=hullwhite_seed)

In [76]:
hullwhite_paths, _, _ = hullwhite.simulate_paths(**hullwhite_params)
payoffs = hullwhite_paths

In [77]:
prices = asianPayoff(payoffs)
prices

array([6.33057037, 6.40023932, 6.37595713, 6.32474915, 6.31353483,
       6.37463307, 6.30025867, 6.31200737, 6.28925178, 6.26430336,
       6.29827663, 6.26328668, 6.29215938, 6.26486319, 6.35008527,
       6.27759942, 6.33549516, 6.34921802, 6.31743636, 6.26255873,
       6.26848975, 6.31656223, 6.28547231, 6.32479423, 6.25131481])

In [ ]:
gbm_signature_pricer.coeffs_.shape

(364,)

In [ ]:
gbm_signature_pricer.predict(prices)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 25 is different from 364)